# Лабораторная работа 1

In [87]:
import pandas as pd
import datetime
cascresto=pd.read_csv('c:/Users/user/Downloads/resto-asap2018/resto-asap2018/casc-resto.csv', sep=';',decimal=",")
cascconstant=pd.read_csv('c:/Users/user/Downloads/resto-asap2018/resto-asap2018/CASC_Constant.csv', sep=',')
cascresto['RKDate'] = pd.to_datetime(cascresto['RKDate'])
cascresto.head()

,CustomerID,Restaurant,RKDate,RegionName,BrandsNames,DishCategoryName,Quantity,SummBasic,SummAfterPointsUsage
0,2898197,391,2017-07-16,Москва и Московская область,TGI FRIDAYS,NON ALCOHOL,2,2.0,2.0
1,2903215,43,2015-04-07,Москва и Московская область,IL Патио,NON ALCOHOL,1,5.0,5.0
2,2748887,43,2015-05-22,Москва и Московская область,IL Патио,NON ALCOHOL,1,5.0,5.0
3,2862077,46,2015-03-05,Москва и Московская область,IL Патио,NON ALCOHOL,1,5.0,5.0
4,2862077,46,2015-03-11,Москва и Московская область,IL Патио,NON ALCOHOL,1,5.0,5.0


In [88]:
startdate=datetime.datetime(2017,7,1)
enddate=datetime.datetime(2017,12,31)

In [89]:
group=cascresto.groupby(['CustomerID'])
def build_statistics(record):
    return pd.Series({    
        # Зависимая переменная Y
        'Flag': len(record.loc[(startdate <= record['RKDate']) & (enddate >= record['RKDate'])]) > 0,
        # Количество дней до последнего визита клиента перед зафиксированной датой 2017-07-01
        'Recency': pd.Timedelta(startdate - record[startdate > record['RKDate']]['RKDate'].max()).days,
        # Средняя частота походов клиентов на дату 2017-07-01, измеряется как среднее количество визитов в месяц за время между 2017-07-01 и первым визитом клиента
        'Frequency': record[record['RKDate'] < startdate].groupby([record[record['RKDate'] < startdate]['RKDate'].dt.year, record[record['RKDate'] < startdate]['RKDate'].dt.month]).size().mean(),
        # Средний чек клиента по всем покупкам до 2017-07-01
        'Monetary_value': record[startdate > record['RKDate']]['SummAfterPointsUsage'].mean(),
        # Сколько раз использовалась скидка
        'Times_using_discount': len(record[record['SummBasic']!=record['SummAfterPointsUsage']])
    })


statistics = group.apply(build_statistics)
statistics.head()

,Flag,Recency,Frequency,Monetary_value,Times_using_discount
CustomerID,,,,,
2728046,False,160.0,8.142857,251.789474,3
2728088,False,720.0,17.000000,279.647059,0
2728089,True,15.0,6.600000,398.734848,16
2728095,False,177.0,9.000000,262.737374,42
2728107,True,115.0,5.250000,393.714286,7


In [90]:
cascconstant.head()

,CustomerId,ActivationDate,Age,Sex,SubscribedEmail,SubscribedPush
0,2728183,2015-01-01,21.0,Female,False,True
1,2728198,2015-01-01,21.0,Female,True,True
2,2728306,2015-01-01,21.0,Female,True,True
3,2728178,2015-01-01,22.0,Male,True,True
4,2728322,2015-01-01,22.0,Male,True,True


In [91]:
fulltable = pd.merge(left=statistics, right=cascconstant[['CustomerId','Sex', 'Age']], left_on='CustomerID', right_on='CustomerId', how='right')
fulltable.head()

,Flag,Recency,Frequency,Monetary_value,Times_using_discount,CustomerId,Sex,Age
0,False,160.0,8.142857,251.789474,3,2728046,Female,24.0
1,False,720.0,17.000000,279.647059,0,2728088,Male,46.0
2,True,15.0,6.600000,398.734848,16,2728089,Female,27.0
3,False,177.0,9.000000,262.737374,42,2728095,Female,54.0
4,True,115.0,5.250000,393.714286,7,2728107,Male,48.0


# Разбиение на обучающую и тестовую выборку

In [92]:
from sklearn.model_selection import train_test_split
fulltable=fulltable.dropna()
train, test = train_test_split(fulltable, test_size=0.2)
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1760 entries, 4201 to 4231
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Flag                  1760 non-null   bool   
 1   Recency               1760 non-null   float64
 2   Frequency             1760 non-null   float64
 3   Monetary_value        1760 non-null   float64
 4   Times_using_discount  1760 non-null   int64  
 5   CustomerId            1760 non-null   int64  
 6   Sex                   1760 non-null   object 
 7   Age                   1760 non-null   float64
dtypes: bool(1), float64(4), int64(2), object(1)
memory usage: 111.7+ KB


In [93]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7040 entries, 7638 to 3170
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Flag                  7040 non-null   bool   
 1   Recency               7040 non-null   float64
 2   Frequency             7040 non-null   float64
 3   Monetary_value        7040 non-null   float64
 4   Times_using_discount  7040 non-null   int64  
 5   CustomerId            7040 non-null   int64  
 6   Sex                   7040 non-null   object 
 7   Age                   7040 non-null   float64
dtypes: bool(1), float64(4), int64(2), object(1)
memory usage: 446.9+ KB


# Построение моделей

In [94]:
from sklearn.linear_model import LogisticRegression
X_train = train[['Recency', 'Frequency', 'Monetary_value']]
Y_train = train.Flag 
X_test = test[['Recency', 'Frequency', 'Monetary_value']]
Y_test = test.Flag 
lr = LogisticRegression()
lr.fit(X_train, Y_train)
Y_pred = logreg.predict(X_test)
from sklearn.metrics import recall_score, precision_score
print('Score: ' + str(logreg.score(X_test, Y_test)))
print('Precision: ' + str(precision_score(Y_pred, Y_test)))
print('Recall: ' + str(recall_score(Y_pred, Y_test)))

Score: 0.7426136363636363
Precision: 0.8086185044359949
Recall: 0.6787234042553192
